# Ajuste fino sobre un modelo de lenguaje enmascarado (PyTorch)

Instalación de librerias de Transformers, Datasets, y Evaluate.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

Necitas configurar tu git, escribir tu correo y nombre en la siguiente celda.

In [ ]:
!git config --global user.email "vg055@hotmail.com"
!git config --global user.name "Victor055"

Usamos el modelo RoBERTa-base-bne que esta basado en RoBERTa, este modelo ha sido pre-entrenado con  un gran conjunto de datos en español con 570 GB de textos limpios y deduplicados. Los datos obtenidos por la libreria nacional de españa entre 2009 al 2019.

In [ ]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "BSC-TeMU/roberta-base-bne"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Para mostrar el número de parámetros de este modelo se utiliza num_parameters().

In [ ]:
roberta_num_parameters = model.num_parameters() / 1_000_000
print(f"'>>> BSC-TeMU/roberta-base-bne number of parameters: {round(roberta_num_parameters)}M'")
print(f"'>>> BERT number of parameters: 110M'")

'>>> BSC-TeMU/roberta-base-bne number of parameters: 125M'
'>>> BERT number of parameters: 110M'


A continuación se muestran los tipos de tokens que predice este modelo con algunos ejemplos.

In [ ]:
#text = "Tengo ganas de <mask>."
text = "imaginate en una <mask> muy grande"

Como humanos, podemos imaginar muchas posibilidades para el token [MASK], como "day", "ride" o "painting". Para los modelos preentrenados, las predicciones dependen del corpus en el que se entrenó el modelo, ya que aprende a captar los patrones estadísticos presentes en los datos. Los datos con los que fue pre-entrenado RoBERTa-base-bne presentaran preferencias en las predicciones de [MASK] porque representan el dominio del dataset. Para predecir la máscara, necesitamos el tokenizador del modelo para producir las entradas para del mismo, así que descarguemos eso también.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

El texto ejemplo se tokeniza y se envia al modelo seleccionado, se extraen los logits y se muestran el top 5 de los predicciones con los más altos logits.

In [ ]:
import torch

inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits
#Encuentra la localización de [MASK] y extrae sus logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Toma las [MASK] candidatas con los logits más altos
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> imaginate en una  web muy grande'
'>>> imaginate en una  forma muy grande'
'>>> imaginate en una  imagen muy grande'
'>>> imaginate en una  foto muy grande'
'>>> imaginate en una  ciudad muy grande'


Podemos ver en los resultados que las predicciones del modelo. Veamos cómo podemos cambiar este dominio con los textos de conjunto de datos del TASS

**Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import Dataset, load_dataset, DatasetDict
import pandas as pd
#Lectura del archivo donde se encuentran los datos de entrenamiento
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tutoriales_MeIA/corpusTASS-2020/train.tsv',sep='\t')
#Unicamente utilizamos el texto y las demás columnas se eliminan
data = data.drop(columns=['id','etiqueta','pais'])
print(data.head(5),"\n")

#Se convierte a un datasetDict
ds_train = Dataset.from_pandas(data)
ds_dict = {'train' : ds_train}
dataset = DatasetDict(ds_dict)
dataset


                                               texto
0  @morbosaborealis jajajaja... eso es verdad... ...
1  @Adriansoler espero y deseo que el interior te...
2  comprendo que te molen mis tattoos, pero no te...
3  Mi última partida jugada, con Sona support. La...
4  Tranquilos que con el.dinero de Camacho seguro... 



DatasetDict({
    train: Dataset({
        features: ['texto'],
        num_rows: 4802
    })
})

Echemos un vistazo a algunos ejemplos para tener una idea de qué tipo de texto estamos tratando. Encadenaremos las funciones Dataset.shuffle() y Dataset.select() para crear una muestra aleatoria:

In [ ]:
sample = dataset["train"].shuffle(seed=42).select(range(3))

for row in sample:
    print(f"\n'>>> Review: {row['texto']}'")


'>>> Review: @soyrosalet @SwarmApp Linda noche Ross extraño escuchar tu voz saludos desde Playas de Rosarito Baja California'

'>>> Review: Es triste quitar todas las cosas de Navidad'

'>>> Review: @MonumentalCR Buenos días, un éxito su maratonica, yo me sumé y sé que van a llevar mucha alegría a personas que están sufriendo'


**Pre-procesamiento de los datos**

Tanto para el modelado de lenguaje autorregresivo como enmascarado, un paso de preprocesamiento común es concatenar todas las instancias y luego dividir todo el corpus en partes de igual tamaño. Esto es bastante diferente de nuestro enfoque habitual, en el que simplemente tokenizamos ejemplos individuales.

¿Por qué concatenar todo junto? La razón es que los ejemplos individuales pueden quedar truncados si son demasiado largos, y eso podría resultar en la pérdida de información que podría ser útil para la tarea de modelado del lenguaje. Entonces, para comenzar, primero tokenizaremos nuestro corpus como de costumbre, pero sin configurar la opción truncation=True en nuestro tokenizador. También tomaremos las identificaciones de las palabras si están disponibles, ya que las necesitaremos más adelante para enmascarar palabras completas.

In [ ]:
def tokenize_function(examples):
    result = tokenizer(examples["texto"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = dataset.map(
    tokenize_function, batched=True, remove_columns=["texto"]
)
tokenized_datasets

Map:   0%|          | 0/4802 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 4802
    })
})

Podemos ver que los textos codificados consisten en input_ids y atencion_mask, así como los word_ids que agregamos.

Ahora que hemos tokenizado nuestros tuits, el siguiente paso es agruparlos todos y dividir el resultado en partes. Pero, ¿qué tan grandes deben ser estos trozos? En última instancia, esto estará determinado por la cantidad de memoria GPU que tenga disponible, pero un buen punto de partida es ver cuál es el tamaño de contexto máximo del modelo. Esto se puede inferir al inspeccionar el atributo model_max_length del tokenizador:

In [ ]:
tokenizer.model_max_length

512

En este caso podemos ver que el tamaño del contexto es de 512 tokens, al igual que con BERT. Entonces, para ejecutar nuestros experimentos en GPU como las que se encuentran en Google Colab, elegiremos un tamaño del contexto más pequeño.

In [ ]:
chunk_size = 128

Para mostrar cómo funciona la concatenación, tomemos algunos tuits de nuestro conjunto de entrenamiento previamente tokenizado y se impremen la cantidad de tokens por cada tuit.

In [ ]:
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Tuit {idx} longitud: {len(sample)}'")

'>>> Tuit 0 longitud: 21'
'>>> Tuit 1 longitud: 20'
'>>> Tuit 2 longitud: 23'


Entonces podemos concatenar todos estos ejemplos, de la siguiente manera:

In [ ]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Longitud de tuits concatenados: {total_length}'")

'>>> Longitud de tuits concatenados: 64'


Genial, se verifica la longitud total, así que ahora dividamos las revisiones concatenadas en partes del tamaño dado por chunk_size. Para hacerlo, iteramos sobre las funciones en concatenated_examples y usamos una lista de comprensión para crear segmentos de cada función. El resultado es un diccionario de fragmentos para cada característica:

In [ ]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 64'


Como puedes ver en este ejemplo, el último fragmento generalmente será más pequeño que el tamaño máximo del fragmento. Hay dos estrategias principales para lidiar con esto:

Suelta el último fragmento si es más pequeño que chunk_size.
Rellene el último trozo hasta que su longitud sea igual a chunk_size.

In [ ]:
def group_texts(examples):
    # Concatena todos los textos
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Calcula la longitud de los textos concatenados
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # Soltamos el último fragmento si es más pequeño que chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Dividir por partes de max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Crear una nueva columna de etiquetas
    result["labels"] = result["input_ids"].copy()
    return result

Tenga en cuenta que en el último paso de group_texts() creamos una nueva columna de etiquetas que es una copia de la de input_ids. Como veremos en breve, eso se debe a que en el modelado de lenguaje enmascarado, el objetivo es predecir tokens enmascarados aleatoriamente en el lote de entrada, y al crear una columna de etiquetas, proporcionamos la verdad básica para que nuestro modelo de lenguaje aprenda.

Ahora apliquemos group_texts() a nuestros conjuntos de datos tokenizados usando nuestra función Dataset.map():

In [ ]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/4802 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 944
    })
})

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' no, pero si estará jugable en el TGS no creo que tarde demasiado. </s><s>@ragnomuelle Yo a veces hecho de menos mi pelo largo por eso mismo entre otras cosas </s><s>A mí nunca me podrán hacer una broma porque no cojo llamadas y menos cuando son ocultas </s><s>#feliz septiembre..es bonito retarse..es increible lo mucho que puedes aprender.. medirse con el obstáculo..eres la joyita de la corona</s><s>Este año el Madrid hará triplete y si lo hace me rapo al cero  grabad el mensaje jejej #ChiringuitoPiqué</s><s>Es conveniente'

Como se esperaba de nuestra función group_texts() anterior, esto se ve idéntico a los input_ids decodificados, pero entonces, ¿cómo es posible que nuestro modelo aprenda algo? Nos falta un paso clave: ¡insertar tokens [MASK] en posiciones aleatorias en las entradas! Veamos cómo podemos hacer esto sobre la marcha durante el ajuste fino usando un recopilador de datos especial.

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



'>>> <s>@morbosaborealis jajajaja... eso es<mask>... aquí<mask> hay uno cuerdo </s><s>@Adriansoler espero y deseo que el interior te cause<mask> mismo </s><s>comprendo<mask> te molen<mask> tattoos, pero no te los hagas 'iguales' </s><s>Mi últimaumbres jugada, con Sona support. La grandes razones<mask> jugar Sona https://<mask>.co/2rgmMeAFIs</s><s>Tranquilos que con el.dinero de<mask> seguro mañana empiezan<mask> escabadoras en el Arra ventilanal </s><s>@daniacal aún'

'>>>  no,<mask> si estará jugable<mask> el TGS no creo que tarde demasiado. </s><s>@ra<mask><mask>uelle Yo a veces<mask> de menos<mask> pelo largo por eso mismo entre otras cosas </s><s>A mí<mask> me podrán hacer una broma porque no cojo llamadas y menos cuando son ocultas </s><s>#feliz septiembre..es bonito retarse..es<mask> lo mucho<mask><mask> aprender.. medirse con el obstáculo..eres la joyita<mask><mask> corona</s><s>Este<mask> el Madrid hará triplete<mask> si lo hace me rapo al cero<mask> grabad el mensaje jejej #<

In [ ]:
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

In [ ]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> <s>@morbosaborealis jajajaja... eso es verdad... aquí no hay<mask> cuerdo </s><s>@<mask><mask><mask><mask><mask> espero y deseo que<mask> interior te<mask><mask><mask> mismo </s><s><mask><mask> que<mask> molen mis tattoos, pero no<mask> los hagas 'iguales' </s><s>Mi última partida jugada, con Sona<mask><mask>.<mask><mask> razones para jugar Sona<mask>://t.<mask>/<mask><mask><mask><mask><mask><mask><mask></s><s>Tranquilos que con el.<mask><mask><mask> Camacho seguro mañana empiezan las escabadoras en el Arraijanal </s><s>@daniacal aún'

'>>>  no, pero si estará jugable en el<mask><mask> no creo<mask> tarde demasiado. </s><s>@<mask><mask><mask><mask><mask> a veces hecho de<mask><mask><mask><mask> por eso mismo<mask> otras cosas </s><s>A mí nunca me podrán<mask><mask> broma porque no<mask> llamadas y menos cuando<mask> ocultas<mask></s><s>#feliz septiembre..es bonito retarse..es increible<mask> mucho que puedes aprender<mask><mask> con el<mask><mask>eres la<mask><mask> de la corona<

In [ ]:
lm_datasets["train"]

Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 944
})

In [ ]:
train_size = 900
test_size = 44

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 900
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 44
    })
})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import TrainingArguments

batch_size = 64
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-Tass2020",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Cloning https://huggingface.co/vg055/roberta-base-bne-finetuned-Tass2020 into local empty directory.


Download file pytorch_model.bin:   0%|          | 16.5k/476M [00:00<?, ?B/s]

Download file runs/Jun22_10-40-13_10e2800807c1/events.out.tfevents.1687430435.10e2800807c1.5726.0: 100%|######…

Download file runs/Jun22_10-40-13_10e2800807c1/events.out.tfevents.1687430463.10e2800807c1.5726.1: 100%|######…

Clean file runs/Jun22_10-40-13_10e2800807c1/events.out.tfevents.1687430463.10e2800807c1.5726.1: 100%|#########…

Clean file runs/Jun22_10-40-13_10e2800807c1/events.out.tfevents.1687430435.10e2800807c1.5726.0:  17%|#7       …

Download file training_args.bin: 100%|##########| 3.87k/3.87k [00:00<?, ?B/s]

Clean file training_args.bin:  26%|##5       | 1.00k/3.87k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/476M [00:00<?, ?B/s]

In [ ]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 137.04


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.951000,3.472807
2,3.371500,2.996654
3,3.131000,3.155004


TrainOutput(global_step=45, training_loss=3.4697325812445747, metrics={'train_runtime': 9.5531, 'train_samples_per_second': 282.632, 'train_steps_per_second': 4.711, 'total_flos': 177704181504000.0, 'train_loss': 3.4697325812445747, 'epoch': 3.0})

In [ ]:
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 23.22


In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/476M [00:00<?, ?B/s]

Upload file runs/Jun22_10-40-13_10e2800807c1/events.out.tfevents.1687430435.10e2800807c1.5726.0:   0%|        …

Upload file runs/Jun22_10-40-13_10e2800807c1/events.out.tfevents.1687430463.10e2800807c1.5726.1:   0%|        …

Upload file training_args.bin:   0%|          | 1.00/3.87k [00:00<?, ?B/s]

To https://huggingface.co/vg055/roberta-base-bne-finetuned-Tass2020
   ba3a77e..633053d  main -> main

   ba3a77e..633053d  main -> main

To https://huggingface.co/vg055/roberta-base-bne-finetuned-Tass2020
   633053d..5b20920  main -> main

   633053d..5b20920  main -> main



'https://huggingface.co/vg055/roberta-base-bne-finetuned-Tass2020/commit/633053d8864445afe008fab78abb5f167560964f'

In [ ]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model="vg055/roberta-base-bne-finetuned-Tass2020"
)

In [ ]:
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> imaginate en una pantalla muy grande
>>> imaginate en una pared muy grande
>>> imaginate en una sala muy grande
>>> imaginate en una escala muy grande
>>> imaginate en una ventana muy grande
